In [1]:
import warnings
warnings.filterwarnings('ignore')

_EPSILON = 1e-08

import numpy as np
import pandas as pd
import random
import os,sys
import pickle


from sklearn.model_selection import train_test_split
from sksurv.metrics import concordance_index_ipcw, concordance_index_censored, brier_score


from models.benchmarks import LANDMARKING
from helpers.data_loader import *


# sys.path.append('../..')
# import utils_network as utils
# from utils_eval import weighted_c_index, weighted_brier_score

In [2]:
OUT_ITERATION               = 5
MICE_IMPUTED_DATA_VERSION   = False
data_mode                   = 'STRATCANS_v1_new2'\
    + ('_MICE' if MICE_IMPUTED_DATA_VERSION else '')
landmarking_mode            = 'CoxPH' #{CoxPH, RSForest}

seed                        = 1234

(data_xs, data_xt, data_time, data_y, data_tte), \
(feat_static, feat_timevarying), \
(xt_bin_list, xt_con_list) = \
import_dataset_STRATCANS_v1(mice_version=MICE_IMPUTED_DATA_VERSION)
    
data_xs           = data_xs[:, [0,1,2,3,4,5,6]]
feat_static       = feat_static[[0,1,2,3,4,5,6]]

data_xt           = data_xt[:, :, [0,1,2,3,6,7,8,9,10]]
feat_timevarying  = feat_timevarying[[0,1,2,3,6,7,8,9,10]]
xt_con_list       = [0, 1, 2, 3, 4, 5, 6, 7]

x_dim_static      = len(feat_static)
x_dim_timevarying = len(feat_timevarying) # this includes delta

max_length                  = np.shape(data_time)[1]
num_Event                   = len(np.unique(data_y)) - 1  #number of next outcome events

In [3]:
hyperparameters = dict()

if landmarking_mode == 'CoxPH':
    hyperparameters['alpha'] = 0.001
elif landmarking_mode == 'RSForest':
    hyperparameters['n_estimators'] =100
else:
    raise ValueError("Wrong survival model. Select {'CoxPH', 'RSForest'}")

In [4]:
LANDMARKING_TIMES = [0, 365, 365*2, 365*3]

PRED_TIMES = [0, 365, 365*2, 365*3]
EVAL_TIMES = [365, 365*2, 365*3, 365*4, 365*5, 365*6, 365*7, 365*8, 365*9, 365*10]
# EVAL_TIMES = [365, 365*2, 365*3, 365*4, 365*5, 365*6, 365*7, 365*8, 365*9, 365*10]


FINAL_RESULT1 = np.zeros([OUT_ITERATION, len(PRED_TIMES), len(EVAL_TIMES)])
FINAL_RESULT2 = np.zeros([OUT_ITERATION, len(PRED_TIMES), len(EVAL_TIMES)])

FINAL_RESULT1_pred = np.zeros([OUT_ITERATION, len(PRED_TIMES), len(EVAL_TIMES)])
FINAL_RESULT2_pred = np.zeros([OUT_ITERATION, len(PRED_TIMES), len(EVAL_TIMES)])

# for out_itr in range(1):
for out_itr in range(OUT_ITERATION):
    
    print('===============LANDMARKING-{} | OUT ITERATION: {}===================='.format(landmarking_mode, out_itr))

    (tr_data_s,te_data_s, tr_data_t,te_data_t, tr_time,te_time, tr_tte,te_tte, tr_label,te_label) = train_test_split(
        data_xs, data_xt, data_time, data_tte, data_y, test_size=0.2, random_state=seed+out_itr
    ) 

    save_path = './{}/Landmarking/{}/itr{}'.format(data_mode, landmarking_mode, out_itr)

    if not os.path.exists(save_path + '/models/'):
        os.makedirs(save_path + '/models/')

    if not os.path.exists(save_path + '/results/'):
        os.makedirs(save_path + '/results/')

    ##### TRAINING
    landmarking = LANDMARKING(LANDMARKING_TIMES=LANDMARKING_TIMES, LANDMARKING_MODE=landmarking_mode, parameters=hyperparameters)
    landmarking.train(tr_data_s, tr_data_t, tr_time, tr_tte, tr_label)
    

    with open(save_path + '/models/landmarking_model.pkl', 'wb') as out_path:
        pickle.dump(landmarking, out_path)


    ##### TESTING
    with open(save_path + '/models/landmarking_model.pkl', 'rb') as in_path:
        landmarking = pickle.load(in_path)    
    te_pred = landmarking.predict(te_data_s, te_data_t, te_time, EVAL_TIMES)


    RESULT1 = -1. * np.ones([len(PRED_TIMES), len(EVAL_TIMES)])
    RESULT2 = -1. * np.ones([len(PRED_TIMES), len(EVAL_TIMES)])

    for p_idx, pred_time in enumerate(PRED_TIMES):
        p_idx_te  = te_tte >= pred_time
        p_idx_tr  = tr_tte >= pred_time

        tmp_pred = landmarking.predict_predtime(te_data_s, te_data_t, te_time, pred_time, EVAL_TIMES)

        tmp_y = tr_label[p_idx_tr]
        tmp_t = tr_tte[p_idx_tr] - pred_time

        tr_y_structured =  [(tmp_y[i], tmp_t[i]) for i in range(len(tmp_y))]
        tr_y_structured = np.array(tr_y_structured, dtype=[('status', 'bool'),('time','<f8')])

        tmp_y = te_label[p_idx_te]
        tmp_t = te_tte[p_idx_te] - pred_time

        te_y_structured =  [(tmp_y[i], tmp_t[i]) for i in range(len(tmp_y))]
        te_y_structured = np.array(te_y_structured, dtype=[('status', 'bool'),('time','<f8')])


        for e_idx, eval_time in enumerate(EVAL_TIMES):
            if np.sum((tmp_t<=eval_time) & (tmp_y==1)) > 0:
                RESULT1[p_idx, e_idx] = concordance_index_ipcw(tr_y_structured, te_y_structured, tmp_pred[p_idx_te][:, e_idx], tau=eval_time)[0]
                RESULT2[p_idx, e_idx] = brier_score(tr_y_structured, te_y_structured, 1.- tmp_pred[p_idx_te][:, e_idx], times=eval_time)[1][0]

    pd.DataFrame(RESULT1, index=PRED_TIMES, columns=EVAL_TIMES).to_csv(save_path+'/results/c_index.csv')
    pd.DataFrame(RESULT2, index=PRED_TIMES, columns=EVAL_TIMES).to_csv(save_path+'/results/brier_score.csv')
    
    FINAL_RESULT1[out_itr, :, :] = RESULT1
    FINAL_RESULT2[out_itr, :, :] = RESULT2
    
    
    RESULT1 = -1. * np.ones([len(PRED_TIMES), len(EVAL_TIMES)])
    RESULT2 = -1. * np.ones([len(PRED_TIMES), len(EVAL_TIMES)])

    for p_idx, pred_time in enumerate(PRED_TIMES):
        p_idx_te  = te_tte >= 0
        p_idx_tr  = tr_tte >= 0

        tmp_pred = landmarking.predict_predtime(te_data_s, te_data_t, te_time, pred_time, EVAL_TIMES)

        tmp_y = tr_label[p_idx_tr]
        tmp_t = tr_tte[p_idx_tr] - 0

        tr_y_structured =  [(tmp_y[i], tmp_t[i]) for i in range(len(tmp_y))]
        tr_y_structured = np.array(tr_y_structured, dtype=[('status', 'bool'),('time','<f8')])

        tmp_y = te_label[p_idx_te]
        tmp_t = te_tte[p_idx_te] - 0

        te_y_structured =  [(tmp_y[i], tmp_t[i]) for i in range(len(tmp_y))]
        te_y_structured = np.array(te_y_structured, dtype=[('status', 'bool'),('time','<f8')])

        for e_idx, eval_time in enumerate(EVAL_TIMES):
            if np.sum((tmp_t<=eval_time) & (tmp_y==1)) > 0:
                RESULT1[p_idx, e_idx] = concordance_index_ipcw(tr_y_structured, te_y_structured, tmp_pred[p_idx_te][:, e_idx], tau=eval_time)[0]
                RESULT2[p_idx, e_idx] = brier_score(tr_y_structured, te_y_structured, 1.- tmp_pred[p_idx_te][:, e_idx], times=eval_time)[1][0]

    pd.DataFrame(RESULT1, index=PRED_TIMES, columns=EVAL_TIMES).to_csv(save_path+'/results/c_index_pred.csv')
    pd.DataFrame(RESULT2, index=PRED_TIMES, columns=EVAL_TIMES).to_csv(save_path+'/results/brier_score_pred.csv')
    
    FINAL_RESULT1_pred[out_itr, :, :] = RESULT1
    FINAL_RESULT2_pred[out_itr, :, :] = RESULT2
    
FINAL_RESULT1[FINAL_RESULT1 == -1] = np.nan
FINAL_RESULT2[FINAL_RESULT2 == -1] = np.nan

pd.DataFrame(np.nanmean(FINAL_RESULT1, axis=0), index=PRED_TIMES, columns=EVAL_TIMES).to_csv('./{}/Landmarking/{}/final_c_index_mean.csv'.format(data_mode, landmarking_mode))
pd.DataFrame(np.nanmean(FINAL_RESULT2, axis=0), index=PRED_TIMES, columns=EVAL_TIMES).to_csv('./{}/Landmarking/{}/final_brier_score_mean.csv'.format(data_mode, landmarking_mode))

pd.DataFrame(np.nanstd(FINAL_RESULT1, axis=0), index=PRED_TIMES, columns=EVAL_TIMES).to_csv('./{}/Landmarking/{}/final_c_index_std.csv'.format(data_mode, landmarking_mode))
pd.DataFrame(np.nanstd(FINAL_RESULT2, axis=0), index=PRED_TIMES, columns=EVAL_TIMES).to_csv('./{}/Landmarking/{}/final_brier_score_std.csv'.format(data_mode, landmarking_mode))


FINAL_RESULT1_pred[FINAL_RESULT1_pred == -1] = np.nan
FINAL_RESULT2_pred[FINAL_RESULT2_pred == -1] = np.nan

pd.DataFrame(np.nanmean(FINAL_RESULT1_pred, axis=0), index=PRED_TIMES, columns=EVAL_TIMES).to_csv('./{}/Landmarking/{}/final_c_index_pred_mean.csv'.format(data_mode, landmarking_mode))
pd.DataFrame(np.nanmean(FINAL_RESULT2_pred, axis=0), index=PRED_TIMES, columns=EVAL_TIMES).to_csv('./{}/Landmarking/{}/final_brier_score_pred_mean.csv'.format(data_mode, landmarking_mode))

pd.DataFrame(np.nanstd(FINAL_RESULT1_pred, axis=0), index=PRED_TIMES, columns=EVAL_TIMES).to_csv('./{}/Landmarking/{}/final_c_index_pred_std.csv'.format(data_mode, landmarking_mode))
pd.DataFrame(np.nanstd(FINAL_RESULT2_pred, axis=0), index=PRED_TIMES, columns=EVAL_TIMES).to_csv('./{}/Landmarking/{}/final_brier_score_pred_std.csv'.format(data_mode, landmarking_mode))

===============LANDMARKING-CoxPH | OUT ITERATION: 0====================
Training 1-th Landmarking Model at 0 - #468
Training 2-th Landmarking Model at 365 - #426
Training 3-th Landmarking Model at 730 - #366
Training 4-th Landmarking Model at 1095 - #306
===============LANDMARKING-CoxPH | OUT ITERATION: 1====================
Training 1-th Landmarking Model at 0 - #468
Training 2-th Landmarking Model at 365 - #428
Training 3-th Landmarking Model at 730 - #360
Training 4-th Landmarking Model at 1095 - #307
===============LANDMARKING-CoxPH | OUT ITERATION: 2====================
Training 1-th Landmarking Model at 0 - #468
Training 2-th Landmarking Model at 365 - #429
Training 3-th Landmarking Model at 730 - #364
Training 4-th Landmarking Model at 1095 - #306
===============LANDMARKING-CoxPH | OUT ITERATION: 3====================
Training 1-th Landmarking Model at 0 - #468
Training 2-th Landmarking Model at 365 - #423
Training 3-th Landmarking Model at 730 - #363
Training 4-th Landmarking M

In [5]:
FINAL_RESULT1[FINAL_RESULT1 == -1] = np.nan
FINAL_RESULT2[FINAL_RESULT2 == -1] = np.nan

pd.DataFrame(np.nanmean(FINAL_RESULT1, axis=0), index=PRED_TIMES, columns=EVAL_TIMES).to_csv('./{}/Landmarking/{}/final_c_index.csv'.format(data_mode, landmarking_mode))
pd.DataFrame(np.nanmean(FINAL_RESULT2, axis=0), index=PRED_TIMES, columns=EVAL_TIMES).to_csv('./{}/Landmarking/{}/final_brier_score.csv'.format(data_mode, landmarking_mode))


FINAL_RESULT1_pred[FINAL_RESULT1_pred == -1] = np.nan
FINAL_RESULT2_pred[FINAL_RESULT2_pred == -1] = np.nan

pd.DataFrame(np.nanmean(FINAL_RESULT1_pred, axis=0), index=PRED_TIMES, columns=EVAL_TIMES).to_csv('./{}/Landmarking/{}/final_c_index_pred.csv'.format(data_mode, landmarking_mode))
pd.DataFrame(np.nanmean(FINAL_RESULT2_pred, axis=0), index=PRED_TIMES, columns=EVAL_TIMES).to_csv('./{}/Landmarking/{}/final_brier_score_pred.csv'.format(data_mode, landmarking_mode))